In [11]:
# Import libaries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np
import requests


### Step 1: Create a soup object from the home page

In [12]:
url = 'http://chubbygrub.com/'
res = requests.get(url)
res.status_code


200

In [13]:
soup = BeautifulSoup(res.content, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [15]:
div = soup.find('div', {'class': 'restaurant-buttons'})

In [16]:
div

<div class="restaurant-buttons">
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<div class="form-group has-warning">
<input autofocus="true" class="form-control input-lg" data-content="The results below will narrow down according to your search" data-placement="top" data-title="Type a Restaurant or Category" id="restaurant-field" placeholder="e.g. McDonald's" type="text"/>
</div>
<a class="btn btn-lg btn-primary" href="/restaurants/aw-restaurants">A&amp;W Restaurants</a> <a class="btn btn-lg btn-primary" href="/restaurants/applebees">Applebee's</a> <a class="btn btn-lg btn-primary" href="/restaurants/arbys">Arby's</a> <a class="btn btn-lg btn-primary" href="/restaurants/atlanta-bread-company">Atlanta Bread Company</a> <a class="btn btn-lg btn-primary" href="/restaurants/bojangles-famous-chicken-n-biscuits">Bojangle's Famous Chicken 'n Biscuits</a> <a class="btn btn-lg btn-primary" href="/restaurants/buffalo-wild-wings">Buffalo Wild Wings</a> <a class="btn btn-lg btn-primary" href="/restaurants/bu

In [30]:
restaurants = []

for a in div.find_all('a'):
    restaurant = {}
    restaurant['name'] = a.text
    restaurant['slug'] = a['href'][13:]
    restaurants.append(restaurant)
    
df_basic = pd.DataFrame(restaurants)
    

In [31]:
df_basic

,name,slug
0,A&W Restaurants,aw-restaurants
1,Applebee's,applebees
2,Arby's,arbys
3,Atlanta Bread Company,atlanta-bread-company
4,Bojangle's Famous Chicken 'n Biscuits,bojangles-famous-chicken-n-biscuits
5,Buffalo Wild Wings,buffalo-wild-wings
6,Burger King,burger-king
7,Captain D's,captain-ds
8,Carl's Jr.,carls-jr
9,Charley's Grilled Subs,charleys-grilled-subs


### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

## Getting new urls

In [53]:
foods = []
for slug in df_basic['slug']:
    
    new_url = url + 'restaurants/' + slug + '/'
    res = requests.get(new_url)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', {'id': 'items'})
    restaurant = soup.find('h1')
    
    for item in table.find_all('tr')[1:]:
        food = {}
        food['name'] = item.find('td', {'itemprop':'name'}).text.strip()
        food['calories'] = item.find('td', {'itemprop':'calories'}).text.strip()
        food['carbs'] = item.find('td', {'itemprop':'carbohydrateContent'}).text.strip()
        food['category'] = item.find('a').text.strip()
        food['fat'] = item.find('td', {'itemprop':'fatContent'}).text.strip()
        
        food['restaurant'] = restaurant.text[:-21]
                               
        foods.append(food)
        

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [54]:
food = pd.DataFrame(foods)

In [55]:
food

,calories,carbs,category,fat,name,restaurant
0,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
1,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
2,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
3,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
4,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
5,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
6,350,60,Shakes,10,A&W® Diet Root Beer Float (large),A&W Restaurants
7,170,30,Shakes,5,A&W® Diet Root Beer Float (medium),A&W Restaurants
8,170,30,Shakes,5,A&W® Diet Root Beer Float (small),A&W Restaurants
9,600,92,Shakes,18,A&W® Diet Root Beer Freeze (large),A&W Restaurants


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [56]:
food.to_csv('/Users/richardmason/Documents/restaurants.csv')

In [69]:
food.loc[1253]

calories                                 990
carbs                                    108
category                             Seafood
fat                                       52
name          Fried Shrimp w/ Cocktail Sauce
restaurant                          Chili's 
Name: 1253, dtype: object

In [66]:
food['calories'].sort_values(ascending=False)


1590       < 5
1591      < 10
2150       992
4949       990
1253       990
4337       990
4349       990
2811       990
4540       990
4074       990
1766       990
661        990
1187       990
2681       990
2989       990
3450       990
3382       990
808        990
1669       990
4826       990
3058       990
4827       990
2891       990
2246       989
2961    988.14
3558       988
3563       984
2129       982
2165       981
2166       981
         ...  
981         10
1381        10
2029        10
2859        10
4            0
3            0
1883         0
2            0
1            0
5            0
1149         0
3741         0
3742         0
4395         0
4312         0
596          0
597          0
631          0
632          0
633          0
0            0
1435         0
3740         0
3739         0
3226         0
3225         0
2740         0
1433         0
1434         0
1532         -
Name: calories, Length: 4977, dtype: object